In [1]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec
import matplotlib.ticker as ticker
from cycler import cycle

from gradient_fill import gradient_fill

mpl.use('agg')


/Users/markdean/anaconda/envs/main/lib/python3.6/site-packages/matplotlib/__init__.py:1401: UserWarning:  This call to matplotlib.use() has no effect
because the backend has already been chosen;
matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

  warnings.warn(_use_error_msg)


Ths using matplotlibrc file from 
https://github.com/mpmdean/matplotlibrc
All fields are using latex by default -- this avoid non-uniformity when using latex in some labels. 

In [2]:
import sys
print("Python version is {} \n".format(sys.version))
print("Matplotlib version is {} \n".format(mpl.__version__))
print("Matplotlibrc file is {}\n".format(mpl.matplotlib_fname()))

Python version is 3.6.1 |Continuum Analytics, Inc.| (default, Mar 22 2017, 19:25:17) 
[GCC 4.2.1 Compatible Apple LLVM 6.0 (clang-600.0.57)] 

Matplotlib version is 2.0.0 

Matplotlibrc file is /Users/markdean/.matplotlib/matplotlibrc



In this case, I think smaller markers are better. So set this at the top and copy this between scripts for other figures. 


In [3]:
mpl.rc('lines', markersize=2)
mpl.rc('errorbar', capsize=2)

## Make fake data

In [4]:
def p0(x, h):
    return 2*np.exp(-(x**2)/0.3)

def p1(x, h):
    return np.exp(-((x-h*6)**2)/0.5)


hs = np.linspace(0, 0.5, 6)
Qs = ['({:.1f}, {:.1f})'.format(h,h) for h in hs]
ms = [0.03*np.random.rand(1) for _ in hs]
cs = [0.2 + 0.1*np.random.rand(1) for _ in hs]

x = np.linspace(-1, 5, 25)
xfit = np.linspace(-1, 5, 1000)
peak0s = [p0(xfit, h) for h, m, c in zip(hs, ms, cs)]
peak1s = [p1(xfit, h) for h, m, c in zip(hs, ms, cs)]
bgs = [m*xfit +c for m, c in zip(ms, cs)]

ys = [p0(x, h) + p1(x, h) + (np.random.rand(x.size)-0.5)*0.2 + m*x + c
      for h, m, c in zip(hs, ms, cs)]


## Create a set of colors and a cyler of markers
The default matplotlib colors are often nice to use and can be accessed via
mpl.rcParams['axes.prop_cycle']

## Create figure

* When trying to display multiple fit components, one can sometimes have trouble with overlapping lines. Translucent colors might be a nice options to correct this.

In [5]:
fig, axs = plt.subplots(2, 3, figsize=(6+3/4, 3))

color = 'k'
for ax, y, peak0, peak1, bg, Q in zip(axs.ravel(), ys, peak0s, peak1s, bgs, Qs):
    gradient_fill(xfit, peak0+bg, fill_color='#1f77b4', ax=ax)
    gradient_fill(xfit, peak1+bg, fill_color='#ff7f0e', ax=ax)
    #poly = ax.fill_between(xfit, peak0+bg, bg, color='#1f77b4', lw=0, alpha=0.3)
    #ax.fill_between(xfit, peak1+bg, bg, color='#ff7f0e', lw=0, alpha=0.3)
    ax.plot(xfit, peak0+peak1+bg, '-', alpha=0.5, color=[0.2]*3)
    ax.errorbar(x, y, y*0.1+0.1, fmt='s', color='k')
    ax.set_ylim(0, 3.9)
    ax.text(3, 3, Q)
    ax.xaxis.set_major_locator(ticker.MultipleLocator(2))
    ax.xaxis.set_minor_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_minor_locator(ticker.MultipleLocator(0.5))

[ax.set_yticklabels('') for ax in axs[:,1:].ravel()]
[ax.set_xlabel('Energy loss (eV)') for ax in axs[1].ravel()]
[ax.set_ylabel('I (Arb. units)') for ax in axs[:,0].ravel()]

    
fig.subplots_adjust(wspace=0, hspace=0, left=0.05, right=0.97, bottom=0.15, top=0.95)

fig.savefig('Ex3.png', dpi=600)

Unfortunately pdf output isn't working for this. Hence a png is written. `